In [ ]:
%load_ext autoreload
%autosave 120
%autoreload 2


# Google Drive Extractor Example

Minimal example of using GoogleDriveExtractor.

**Note**: This uses service account authentication via `credentials/drive_service_account.json`. Make sure the service account has access to the Google Drive folders you want to access.


In [ ]:
import sys
sys.path.append('../src')

from google_drive.extractor import GoogleDriveExtractor
from google_drive.config import FIRESIDE_FOLDER_ID

# Initialize extractor
extractor = GoogleDriveExtractor()


In [ ]:
# List files
folder_id = FIRESIDE_FOLDER_ID
files = extractor.list_files(folder_id, recursive=True)
print(f"Found {len(files)} files")


In [ ]:
import pandas as pd
files_df = pd.DataFrame(files)
files_df

In [ ]:
from db.db import SupabaseDB
db = SupabaseDB()
db.get_all_records("files")


In [ ]:
files_df.mimeType.value_counts()

In [ ]:
def file_type(row):
    if row['mimeType'] == "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet":
        return "mis"
    if row['mimeType'] == "application/vnd.openxmlformats-officedocument.wordprocessingml.document":
        if "quarter" in row['name'].lower():
            return "final_report"
        else:
            return "commentary"

def owner_type(row):
    if row['mimeType'] == "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet":
        return "company"
    if row['mimeType'] == "application/vnd.openxmlformats-officedocument.wordprocessingml.document":
        if "quarter" in row['name'].lower():
            return "final_report"
        else:
            return "investment_team"


def owner(row):
    if row['mimeType'] == "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet":
        return "Company CFO"
    if row['mimeType'] == "application/vnd.openxmlformats-officedocument.wordprocessingml.document":
        if "quarter" in row['name'].lower():
            return "Partners"
        else:
            return "Rahul Mehta"

print(len(files_df))
ignore_mime_types = ['application/pdf', 'application/vnd.google-apps.folder']
remove_flag = files_df['mimeType'].isin(ignore_mime_types)
print(f"Removing {remove_flag.sum()} files")
files_df = files_df[~remove_flag].copy()

files_df['file_type'] = files_df.apply(file_type, axis=1)
files_df['owner_type'] = files_df.apply(owner_type, axis=1)
files_df['owner'] = files_df.apply(owner, axis=1)
files_df
import datetime
asof_date_map = {
    "19yYx7qjCADnS8ligMzO6ew7F-_uqmmlF": datetime.date(2025, 6, 30),
    "14vXyasqWexxaMoYATCjBgVu4c8GDTSoh": datetime.date(2025, 3, 31),
}


files_df['asof_date'] = files_df['parents'].apply(lambda x: asof_date_map.get(x[0]))

keep_cols_rename = {
    'name': 'file_name',
    'file_type': 'type',
    'id': 'path'
}
other_keep_cols = ['asof_date', 'owner_type', 'owner']
files_df = files_df[list(keep_cols_rename.keys()) + other_keep_cols].rename(columns=keep_cols_rename)
print(len(files_df))
files_df

In [ ]:
from db.db import SupabaseDB
db = SupabaseDB()
company_df = db.get_all_records("company")
company_df['registered_name_no_pvt_ltd'] = company_df['registered_name'].apply(lambda x: str(x).replace(' Pvt. Ltd.', '').lower())
company_df['lower_name'] = company_df['common_name'].str.lower()

In [ ]:
company_lower_name_map = dict(zip(company_df['lower_name'], company_df['id']))
registered_name_map = dict(zip(company_df['registered_name_no_pvt_ltd'], company_df['id']))
registered_name_map

In [ ]:
company_lower_name_map = dict(zip(company_df['lower_name'], company_df['id']))
def get_company_id(row):
    file_name = row['file_name'].lower()
    company_ids = []
    for key, value in company_lower_name_map.items():
        if key in file_name:
            company_ids.append(value)
    for key, value in registered_name_map.items():
        if key in file_name:
            company_ids.append(value)
    company_ids = list(set(company_ids))
    if len(company_ids) > 1:
        print(f"Multiple companies found for {file_name}")
        return None
    elif len(company_ids) == 1:
        return int(company_ids[0])
    else:
        return None
files_df['company_id'] = files_df.apply(get_company_id, axis=1).fillna(29).astype(int)
files_df

In [ ]:
# # from db.db import SupabaseDB
# db = SupabaseDB()
db.insert_records(files_df, "files", upsert = False,
        add_metadata = False,
        pkey_drop_duplicates = False,
        non_nullable_check = False,)

In [ ]:
from google_drive.config import FILE_TYPE_MAP, GOOGLE_MIME_TYPES
def file_type(mime_type)

In [ ]:
# Read spreadsheet
sheets_data = extractor.read_spreadsheet("1cE0QvVtFKc9jMYxxdvOtSaMm43FLyBza")
print(f"Sheets: {list(sheets_data.keys())}")


In [ ]:
raw_data = sheets_data.get('Financial performance').get('raw_data')
# keep only rows 7-12
raw_data.iloc[6:12]


In [ ]:
processed_data = sheets_data.get('Financial performance').get('processed_data')
processed_data.iloc[6:12]


In [ ]:
merged_cells = sheets_data.get('merged_cells')
merged_cells.get('Financial performance')

In [ ]:
# Read document
tables_data = extractor.read_document("1MDNfalS1ujwP9UmICDlVA9ib1FFKmiA3")
print(f"Tables: {list(tables_data.keys())}")


In [ ]:
table_0 = tables_data.get('table_0')
table_0

In [ ]:
company_df = db.get_all_records("company")

In [ ]:
company_df = db.get_all_records("company")[['id', 'common_name']]
commentary_files_df = db.get_all_records("files", simple_filter = {'type': "commentary"})
commentary_files_df = commentary_files_df.merge(company_df, left_on = 'company_id', right_on = 'id', how = 'left')
dfs = []
for index, row in commentary_files_df.iterrows():
    print(row)
    file_name = row['file_name']
    tables_data = extractor.read_document(row['path'])
    print(f"Tables: {list(tables_data.keys())}")
    if len(tables_data.keys()) == 0:
        print(f"No tables found for {file_name}")
        continue
    if len(tables_data.keys()) > 1:
        print(f"Multiple tables found for {file_name}, skipping")
        continue
    table_0 = tables_data.get('table_0')
    table_0['file_id'] = row['id_x']
    table_0['company_id'] = row['company_id']

    dfs.append(table_0)
def _quarter(row):
    try:
        return f"{row['Quarter']} {row['Year']}"
    except:
        return None
commentary_tables_df = pd.concat(dfs)
commentary_tables_df['quarter'] = commentary_tables_df.apply(_quarter, axis=1)
commentary_tables_df['commentary'] = commentary_tables_df['Commentary'].str.strip()
commentary_tables_df



In [ ]:
keep_cols = ['file_id', 'company_id', 'quarter', 'commentary']
db.insert_records(commentary_tables_df[keep_cols], "commentary", upsert = False,
        add_metadata = False,
        pkey_drop_duplicates = False,
        non_nullable_check = False,)


In [ ]:
# Read spreadsheet
sheets_data = extractor.read_spreadsheet("1cE0QvVtFKc9jMYxxdvOtSaMm43FLyBza")
raw_share_capital_df = sheets_data.get('Share capital table').get('raw_data')
raw_share_capital_df

In [ ]:
# replace "Other Investor" with "Other" in the category column
all_melted_df['category'] = all_melted_df['category'].str.replace("Other Investor", "Other")
all_melted_df

In [ ]:
db.insert_records(all_melted_df, "share_cap_table", upsert = False,
        add_metadata = False,
        pkey_drop_duplicates = False,
        non_nullable_check = False,)


In [ ]:
!git status

In [ ]:
allowed_categories = ['Other', 'Founder', 'Fireside', 'ESOP']
def get_category(category):
    if category in allowed_categories:
        return category
    return 'Other'



dfs = []

def next_empty_row_finder(df, row_number=0, end_col=None):
    # Find the next empty row in the dataframe, after the given row number
    if end_col is not None:
        df = df.iloc[:, :end_col]
    for index, row in df.iterrows():
        if index > row_number and row.isnull().all():
            return index
    return None

def get_share_cap_table_start_row(df):
    # The first column where you find "Share Capital Table", the row after that one is the start of the table
    for index, row in df.iterrows():
        if "Share Capital Table" in str(row[0]):
            return index + 1
    return None

def get_share_cap_table_start_and_end_row(df):
    start_row = get_share_cap_table_start_row(df)
    if start_row is None:
        return None, None
    end_row = next_empty_row_finder(df, start_row)
    return start_row, end_row



company_df = db.get_all_records("company")[['id', 'common_name']]
cap_table_files_df = db.get_all_records("files", simple_filter = {'type': "mis"})
cap_table_files_df = cap_table_files_df.merge(company_df, left_on = 'company_id', right_on = 'id', how = 'left')
dfs = []
error_files = []
for index, row in cap_table_files_df.iterrows():
    print(row)

    sheets_data = extractor.read_spreadsheet(row['path'])
    share_capital_sheet_names = [sheet_name for sheet_name in sheets_data.keys() if "share" in str(sheet_name).lower()]
    if len(share_capital_sheet_names) == 0:
        print(f"No share capital table found for {row['file_name']}")
        error_files.append(row['file_name'])
        continue
    if len(share_capital_sheet_names) > 1:
        print(f"Multiple share capital tables found for {row['file_name']}")
        error_files.append(row['file_name'])
        continue
    if 'raw_data' not in sheets_data.get(share_capital_sheet_names[0]).keys():
        print(f"No raw data found for {row['file_name']}")
        error_files.append(row['file_name'])
        continue
    raw_share_capital_df = sheets_data.get(share_capital_sheet_names[0]).get('raw_data')

    share_cap_table_start_row, share_cap_table_end_row = get_share_cap_table_start_and_end_row(raw_share_capital_df)


    df = raw_share_capital_df.iloc[share_cap_table_start_row+1:share_cap_table_end_row]
    # raw_share_capital_df.loc[share_cap_table_start_row, 0] = "id"
    df.columns = raw_share_capital_df.iloc[share_cap_table_start_row].apply(lambda x: str(x).strip())
    print(df.columns)

    percentage_cols = [col for col in df.columns if "%" in str(col)]
    nan_cols = [col for col in df.columns if col in ('nan', 'na')]

    date_cols = [col for col in df.columns if "date" in str(col).lower() and col not in percentage_cols and col not in nan_cols]
    name_of_company_cols = [col for col in df.columns if ("company" in str(col).lower() or 'Moxie' in str(col)) and col not in percentage_cols and col not in nan_cols]

    shareholder_name_cols = [col for col in df.columns if "shareholder" in str(col).lower() and col not in percentage_cols and col not in nan_cols]
    category_col = [col for col in df.columns if "category" in str(col).lower() and col not in percentage_cols and col not in nan_cols]
    if len(shareholder_name_cols) == 0:
        print(f"No shareholder name column found for {row['file_name']}")
        error_files.append(row['file_name'])
        continue
    if len(shareholder_name_cols) > 1:
        print(f"Multiple shareholder name columns found for {row['file_name']}")
        error_files.append(row['file_name'])
        continue
    shareholder_name_col = shareholder_name_cols[0]
    if len(category_col) == 0:
        print(f"No category column found for {row['file_name']}")
        error_files.append(row['file_name'])
        continue
    if len(category_col) > 1:
        print(f"Multiple category columns found for {row['file_name']}")
        error_files.append(row['file_name'])
        continue
    category_col = category_col[0]

    print(df.columns)
    df = df.drop(columns=list(set(name_of_company_cols+ date_cols+ nan_cols + percentage_cols))).copy()
    index_cols = [category_col, shareholder_name_col]
    melt_cols = [col for col in df.columns if col not in index_cols]
    melted_df = df.melt(
        id_vars = index_cols,
        value_vars = melt_cols,
        var_name = 'series',
        value_name = 'number_of_shares'
    ).dropna(subset = ['Category'])
    melted_df['number_of_shares'] = melted_df['number_of_shares'].apply(lambda x: 0 if str(x).replace(' ', '') == '-' else x)
    melted_df['number_of_shares'] = melted_df['number_of_shares'].fillna(0).astype(int)
    melted_df = melted_df[melted_df['number_of_shares'] > 0]
    melted_df['category'] = melted_df['Category'].apply(get_category)

    melted_df['shareholder_name'] = melted_df[shareholder_name_col].str.strip()
    melted_df['series'] = melted_df['series'].str.strip()
    melted_df['number_of_shares'] = melted_df['number_of_shares'].astype(int)
    melted_df = melted_df.drop(columns=['Category', shareholder_name_col]).copy()
    melted_df['file_id'] = row['id_x']
    melted_df['company_id'] = row['company_id']
    dfs.append(melted_df)

all_melted_df = pd.concat(dfs)
print(error_files)
all_melted_df




In [ ]:
# Other Investor to "Other  "
all_melted_df['category'] = all_melted_df['category'].str.replace("Other Investor", "Other")
db.insert_records(all_melted_df, "share_cap_table", upsert = False,
        add_metadata = False,
        pkey_drop_duplicates = False,
        non_nullable_check = False,)

In [ ]:

def next_empty_row_finder(df, row_number=0, end_col=None):
    # Find the next empty row in the dataframe, after the given row number
    if end_col is not None:
        df = df.iloc[:, :end_col]
    for index, row in df.iterrows():
        if index > row_number and row.isnull().all():
            return index
    return None


def get_particulars_row(df, row_number=0):
    # The first column where you find "Share Capital Table", the row after that one is the start of the table
    for index, row in df.iterrows():
        if index > row_number and "particulars" in str(row[0]).lower():
            return index
    return None

quarter_strings = ['JFM', 'AMJ', 'JAS', 'OND', 'Q1', 'Q2', 'Q3', 'Q4']
year_strings = ['2020', '2021', '2022', '2023', '2024', '2025', '2026', '2027', '2028', '2029', '2030', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30']
inception_strings = ["inceptiontilldate",]
def get_particulars_last_columns(df, partculars_row):
    # find the last column where you find a string can be constructed using any combination of quarter_strings, year_strings, inception_strings
    row = df.iloc[partculars_row]
    count = 0
    for k, item in row.items():
        if 'particulars' in str(item).lower():
            count += 1
            continue

        item_str = str(item).lower().replace(' ', '').replace('\n', '')
        original_item_str_len = len(item_str)
        for quarter in quarter_strings:
            item_str = item_str.replace(quarter.lower(), '')
        for year in year_strings:
            item_str = item_str.replace(year.lower(), '')
        for inception in inception_strings:
            item_str = item_str.replace(inception.lower(), '')
        if len(item_str) < original_item_str_len:
            count += 1
        else:
            break
    return count

def get_all_particulars_tables(df):
    pairs = []
    start_row = get_particulars_row(df)
    if start_row is None:
        return pairs
    end_col = get_particulars_last_columns(df, start_row)
    end_row = next_empty_row_finder(df, start_row, end_col)

    pairs.append({
        "start_row": start_row,
        "end_row": end_row,
        "end_col": end_col,
    })
    while start_row is not None and end_row is not None:
        start_row = get_particulars_row(df, end_row)
        if start_row is not None:
            end_col = get_particulars_last_columns(df, start_row)
            end_row = next_empty_row_finder(df, start_row, end_col)
            if end_row is None:
                end_row = len(df)
            pairs.append(
                {
                    "start_row": start_row,
                    "end_row": end_row,
                    "end_col": end_col,
                }
            )
    return pairs




In [ ]:
errors_files = []
dfs = []
company_df = db.get_all_records("company")[['id', 'common_name']]
cap_table_files_df = db.get_all_records("files", simple_filter = {'type': "mis"})
cap_table_files_df = cap_table_files_df.merge(company_df, left_on = 'company_id', right_on = 'id', how = 'left')
dfs = []
error_files = []
for index, row in cap_table_files_df.iterrows():
    # if 'happi' not in str(row['file_name']).lower():
    #     print(f"Skipping {row['file_name']}")
    #     continue

    sheets_data = extractor.read_spreadsheet(row['path'])
    financial_sheet_names = [sheet_name for sheet_name in sheets_data.keys() if "financial" in str(sheet_name).lower()]
    if len(financial_sheet_names) == 0:
        print(f"No financial sheet found for {row['file_name']}")
        error_files.append(row['file_name'])
        # continue
    if len(share_capital_sheet_names) > 1:
        print(f"Multiple financial sheets found for {row['file_name']}")
        error_files.append(row['file_name'])
        # continue
    if 'processed_data' not in sheets_data.get(financial_sheet_names[0]).keys():
        print(f"No processed data found for {row['file_name']}")
        error_files.append(row['file_name'])
        # continue


    processed_financial_data_df = sheets_data.get(financial_sheet_names[0]).get('processed_data')

    tables = get_all_particulars_tables(processed_financial_data_df)
    if len(tables) != 4:
        print(f"Incorrect number of financial tables found for {row['file_name']}")
        print(tables)
        error_files.append(row['file_name'])
        continue
    first_table = tables[0]
    # processed_financial_data_df.iloc[first_table['start_row']:first_table['end_row'], :first_table['end_col']]
    def remove_identical_adjacent_columns(df):
        drop_cols = []
        for i in range(len(df.columns) - 1):
            col_i = df.columns[i]
            col_j = df.columns[i+1]
            if df[col_i].equals(df[col_j]):
                drop_cols.append(col_j)
        return df.drop(columns=drop_cols)

    df = remove_identical_adjacent_columns(processed_financial_data_df.iloc[first_table['start_row']:first_table['end_row'], :first_table['end_col']])
    df2 = df.iloc[1:].copy()
    df2.columns = df.iloc[0]
    df2 = df2.dropna(axis=1, how='all').copy()
    df2['file_id'] = row['id_x']
    df2['company_id'] = row['company_id']
    dfs.append(df2)

print(error_files)

df2 = pd.concat(dfs)
df2

In [ ]:

index_cols = ['file_id', 'company_id', 'Particulars']
melt_cols = [col for col in df2.columns if col not in index_cols]
melted_df2 = df2.melt(
    id_vars = index_cols,
    value_vars = melt_cols,
    var_name = 'quarter',
    value_name = 'attribute_number'
)
melted_df2


In [ ]:
def get_attribute_type(row):
    # allower attribute_types: financials, headcount
    if "head" in str(row['Particulars']).lower():
        return "headcount"
    return "financials"

def get_attribute_dimension(row):
    # segregate INR Mn for now"
    if "inr mn" in str(row['Particulars']).lower():
        return "INR Mn"
    return None

melted_df2['attribute_type'] = melted_df2.apply(get_attribute_type, axis=1)
melted_df2['attribute_text'] = melted_df2.apply(get_attribute_dimension, axis=1)
melted_df2['attribute_name'] = melted_df2['Particulars']
melted_df2 = melted_df2.drop(columns=['Particulars']).copy()
melted_df2

In [ ]:
from db.db import SupabaseDB
db = SupabaseDB()
db.insert_records(melted_df2, "quarterly_attributes", upsert = False,
        add_metadata = False,
        pkey_drop_duplicates = False,
        non_nullable_check = False,)


In [ ]:
melted_df2[melted_df2['company_id']==9].quarter.value_counts()